# 首次调用API
以下内容来源于deepseek官方文档：https://api-docs.deepseek.com/zh-cn/

In [ ]:
# Please install OpenAI SDK first: `pip3 install openai`
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get('DEEPSEEK_API_KEY'),#为避免硬编码，请在虚拟环境中配置环境变量
    base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",#选择模型
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

Hello! How can I assist you today? 😊


## message
message是一个消息列表，每个消息是一个字典，描述一轮对话中的一句话，格式大致如下：

每条mesage的标准字段：

- id: 消息id，字符串
- content: 消息内容，字符串
- role: 消息角色，字符串，可选值有：user, assistant, system
- created_at: 消息创建时间，时间戳
- updated_at: 消息更新时间，时间戳
- model: 模型名称，字符串
- metadata: 消息元数据，字典
- usage: 消息使用情况，字典


In [ ]:
messages = [
    {"role": "system", "content": "你是一个友好的中文助手。"},#定义模型行为、身份、语气
    {"role": "user", "content": "你好！"},#用户输入
    {"role": "assistant", "content": "你好呀，我能帮你做什么？"}#模型回复
]


## 多轮对话
在多轮对话中，通常就是维护一个message列表：

In [ ]:
user_input = input("你: ")
messages.append({"role": "user", "content": user_input})

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

reply = response.choices[0].message.content
messages.append({"role": "assistant", "content": reply})


对话式应用的核心就是上下文记忆结构，旨在让模型理解现在的问题与之前的对话有关。

## 流式输出
让大模型边生成边输出，而不是全部生成完再返回。


In [3]:
from openai import OpenAI
import os
#示例
client = OpenAI(
    api_key=os.environ.get('DEEPSEEK_API_KEY'),#为避免硬编码，请在虚拟环境中配置环境变量
    base_url="https://api.deepseek.com"
)


response = client.chat.completions.create(
    model="deepseek-chat",#选择模型
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "做一下自我介绍，字数控制在100到200个汉字之间"},
    ],
    stream=True
)

# 逐步接收并处理响应
for chunk in response:
    chunk_message = chunk.choices[0].delta.content
    print(chunk_message, end='', flush=True)

您好！我是您的智能助手，由深度求索公司创造的DeepSeek。我基于先进的大语言模型技术，致力于为您提供准确、有用的信息和帮助。

我擅长文本处理、知识问答、逻辑分析、创意写作等多种任务，支持上传图像、txt、pdf、ppt、word、excel等文件并从中读取文字信息进行处理。虽然我是纯文本模型，但通过文件上传功能，我能更好地理解您的需求。

目前我完全免费使用，拥有128K的上下文处理能力。如果您需要最新信息，记得在Web或App上手动点开联网搜索功能哦！

我会以热情、细腻的方式为您服务，努力做您最可靠的学习伙伴和工作助手。有什么问题尽管问我吧！😊

另一种流式输出的方式：基于 Requests 库的流式输出

In [6]:
import requests
import json
import os

API_KEY = os.environ.get('DEEPSEEK_API_KEY')

# 添加检查API密钥是否存在
if not API_KEY:
    print("❌ 错误: 请先设置 DEEPSEEK_API_KEY 环境变量")
    exit(1)

#修正URL，添加chat/completions端点
url = "https://api.deepseek.com/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}",
}

data = {
    "model": "deepseek-chat",   
    "stream": True,
    "messages": [
        {"role": "system", "content": "你是一名有耐心的AI助手"},
        {"role": "user", "content": "解释一下快速排序算法"}
    ]
}

try:
    with requests.post(url, headers=headers, json=data, stream=True) as response:
        # 检查HTTP状态码
        if response.status_code != 200:
            print(f"❌ HTTP错误: {response.status_code}")
            print(f"响应内容: {response.text}")
        else:
            print("正在接收流式响应...")
            content_received = False
            for line in response.iter_lines():
                if line:
                    decoded_line = line.decode("utf-8")
                    # 调试输出原始行内容
                    # print(f"原始行: {decoded_line}")
                    
                    # 流式返回格式以 "data:" 开头
                    if decoded_line.startswith("data: "):
                        content_received = True
                        payload = decoded_line[6:]
                        if payload == "[DONE]":
                            break
                        # 解析 JSON
                        try:
                            event = json.loads(payload)
                            delta = event["choices"][0]["delta"]
                            if "content" in delta:
                                print(delta["content"], end="", flush=True)
                        except json.JSONDecodeError as e:
                            print(f"\n❌ JSON解析错误: {e}")
                            print(f"问题数据: {payload}")
                    else:
                        # 输出非标准格式的行，便于调试
                        print(f"\n⚠️ 非标准格式行: {decoded_line}")
            
            if not content_received:
                print("⚠️ 未接收到有效的数据内容")

except requests.exceptions.RequestException as e:
    print(f"❌ 请求异常: {e}")

print("\n\n✅ 流式输出结束")


正在接收流式响应...
好的，我会用清晰、易懂的方式为你解释快速排序算法。

### 1. 核心思想：分而治之

快速排序的核心思想是 **“分而治之”** 。它选择一个元素作为“基准”，然后将数组重新排列，使得所有比基准小的元素都放在基准的左边，所有比基准大的元素都放在基准的右边。这个操作被称为 **“分区”**。

完成一次分区后，这个基准元素就已经处于它最终应该在的位置了。然后，算法会递归地对基准左边和右边的两个子数组进行同样的操作，直到整个数组变得有序。

---

### 2. 算法步骤详解

让我们通过一个例子来分解整个过程。假设我们要排序的数组是：`[6, 3, 8, 5, 2, 7, 4]`。

**步骤一：选择基准**

选择一个元素作为基准。选择方法有很多，为了简单，我们通常选择第一个元素、最后一个元素或中间元素。这里我们选择最后一个元素 `4` 作为基准。

**步骤二：分区**

这是算法中最关键的一步。我们的目标是重新排列数组，让所有小于 `4` 的元素在左边，所有大于 `4` 的元素在右边。

我们使用两个指针（或索引）：
- `i` (慢指针)：指向小于基准的子数组的末尾。
- `j` (快指针)：遍历整个数组（除了基准）。

**分区过程：**
1. 初始化 `i = -1`（起始位置的前一个）。
2. 从 `j = 0` 开始遍历到倒数第二个元素（即 `j < 6`）。
3. 如果 `arr[j]` 小于等于基准 `4`，就将 `i` 向右移动一位，然后交换 `arr[i]` 和 `arr[j]`。
4. 遍历结束后，将基准 `arr[high]` 与 `arr[i+1]` 交换。这样，基准就放到了正确的位置。

让我们手动模拟一下：

初始数组：`[6, 3, 8, 5, 2, 7, 4]`，基准 `pivot = 4`
- `i = -1`, `j = 0`: `arr[0] = 6` > 4，不做交换。数组不变。
- `i = -1`, `j = 1`: `arr[1] = 3` <= 4，`i` 变为 0，交换 `arr[0]` 和 `arr[1]`。数组变为：`[3, 6, 8, 5, 2, 7, 4]`
- `i = 0`, `j = 2`: `arr[2] = 8` > 4，不做交换。数组不变。
- `i = 0`, 